# Receives partial offer containing only price

`validate_partial_offer()` calls `is_price_feasible()`:

Buyer case

- Given a **price \(X\)**, assume the **maximum quality \(Q = 100\)** (or maximum demand).

- Compute the buyer’s **maximum profit at price \(X\)** with \(Q = 100\).

- If this maximum profit is **greater than or equal to the Nash profit**:
  - Call `optimal_quality_for_wholesale_price()`

- Otherwise, if the maximum profit **is less than the Nash profit** <span style="color:#e53935">(the term is too unfavourable)</span>:
  - Send the **Nash equilibrium offer** directly.

Supplier Case

- Given a **price \(X\)**, compute the **optimal quality** for the supplier using the formula:
  
  $$
  Q = d_{max} - \frac{production\_cost \times (d_{max} - d_{min})}{X}
  $$

- Compute the **supplier’s maximum profit at price \(X\)** using this quality \(Q\).

- If this maximum profit is **greater than or equal to the Nash profit**:
  - Call `optimal_quality_for_wholesale_price()`

- Otherwise, if the maximum profit **is less than the Nash profit** <span style="color:#e53935">(the term is too unfavourable)</span>:
  - Send the **Nash equilibrium offer** directly.

# Receives partial offer containing only quality

`validate_partial_offer()` calls `is_quality_feasible()`:

Buyer case

- Given a **quality \(X\)**

- Compute the buyer’s **minimum acceptable price** with \(Q = X\) **to reach nash_profit**.

- If the minimum acceptable price is **less than 0** or **greater than the MSP** <span style="color:#e53935">(the term is too unfavourable)</span>:
  - Send the **Nash equilibrium offer** directly.

- Otherwise, if it is between acceptable margins:
  - Call `optimal_wholesale_price_for_quality()`

Supplier Case

- Given a **quality \(X\)**

- Compute the supplier's **maximum acceptable price** with \(Q = X\) **to reach nash_profit**.

- If the maximum acceptable price is **less than the PC** <span style="color:#e53935">(the term is too unfavourable)</span>:
  - Send the **Nash equilibrium offer** directly.

- Otherwise, if it is between acceptable margins:
  - Call `optimal_wholesale_price_for_quality()`

# Partial offer contains easy quality

Adjust the price with `optimal_wholesale_price_for_quality()`
using inverse formula of profit with fixed quality (given)

# Partial offer contains easy wholesale price

Adjust the quality with `optimal_quality_for_wholesale_price()` using inverse formula of profit with fixed price (given).  
This involves solving a quadratic equation derived
from the profits' equations:

$$
\text{target\_profit\_buyer} = (market\_price - wholesale\_price) \times ES(q)
$$ 

$$
\text{target\_profit\_supplier} = (wholesale\_price \times ES(q)) - production\_cost \times q
$$

where 

$$
ES(q) = \frac{(quality^2 - demand\_min^2)}{2} + quality \times \frac{(demand\_max - quality)}{(demand\_max - demand\_min)}
$$

The roots represent points where the profit exactly matches the target.

Next, the vertex (peak) of the profit function is identified—the quality value that maximizes the continuous profit function.  
For the buyer, this is typically at the maximum demand; for the supplier, it’s at a point adjusted by production cost and price.

To select practical integer quality values, the code samples integers around the roots, the vertex, and the demand boundaries,  
ensuring all promising candidates are considered.

Finally, **among candidates that satisfy the bot’s profit being at least the Nash target, the function selects the quality that**  
**maximizes the user’s profit**, aiming for negotiation efficiency. In case of ties, it chooses the quality maximizing the combined
total profit.  
If none satisfy the target, it returns no feasible solution.

